# Legacy Inputs From a GDX File

**Objective**  
Build the EPM CSV input tree from a consolidated `.gdx` snapshot so that older workflows keep running.

**Data requirements (user-provided) and method**  
- Data requirements: The source `.gdx` file with the model symbols, a mapping file that links symbols to legacy CSV layouts, and target directories where the recreated `input/` tree will be written.  
- Method: Load `gdxpds`/pandas helpers, read the mapping, iterate through each symbol in the `.gdx`, convert to DataFrames, and write the expected CSVs (creating empty placeholders when needed) while logging any unmapped content.

**Overview of steps**  
1. Step 1 - Load Python and GDX dependencies.  
2. Step 2 - Configure the source `.gdx`, mapping file, and output folders.  
3. Step 3 - Declare the hard-coded legacy CSV layout.  
4. Step 4 - Load the symbol-to-file mapping.  
5. Step 5 - Read the `.gdx` file into pandas objects.  
6. Step 6 - Define helper functions to reshape and write tables.  
7. Step 7 - Write the legacy CSV tree and any extras.  
8. Step 8 - Review log messages or warnings.


## Step 1 · Load dependencies
Import the required libraries. Ensure `gams.transfer` is available in your environment.


In [25]:

from pathlib import Path
from typing import Optional

import pandas as pd

try:
    import gams.transfer as gt
except ImportError as err:
    raise ImportError("Install the GAMS Python API before running this notebook.") from err



## Step 2 · Configure paths
Keep all exports inside the local `./output` folder. Update the placeholders before continuing.


In [26]:

# TODO: point this to your legacy GDX file
GDX_PATH = Path("input/input_epm_Turkiye_v8.gdx")  # default sample path

# TODO: choose a subfolder inside ./output for the generated CSV files
OUTPUT_BASE = Path("output")
TARGET_FOLDER = "data_test_from_gdx"  # update if you prefer a different name

# Mapping table that translates legacy GDX symbols into the expected CSV names
MAPPING_PATH = Path("input/symbol_mapping.csv")

OVERWRITE = True  # set False to retain existing CSV files

if not MAPPING_PATH.exists():
    raise FileNotFoundError(f"Missing mapping table: {MAPPING_PATH}. Populate it before continuing.")

EXPORT_ROOT = (OUTPUT_BASE / TARGET_FOLDER).resolve()
EXPECTED_ROOT = (Path.cwd() / "output").resolve()
if not GDX_PATH.exists():
    raise FileNotFoundError(f"Update GDX_PATH to point to your legacy file. Missing: {GDX_PATH}")
try:
    EXPORT_ROOT.relative_to(EXPECTED_ROOT)
except ValueError as exc:
    raise ValueError("Choose TARGET_FOLDER inside the ./output directory.") from exc

EXPORT_ROOT.mkdir(parents=True, exist_ok=True)
EXTRAS_ROOT = (EXPORT_ROOT / "extras")
EXTRAS_ROOT.mkdir(parents=True, exist_ok=True)



## Step 3 · Legacy CSV layout (hard-coded)
The structure below mirrors the historical `input_readers.gms` file. No parsing is performed at runtime.


In [27]:
CSV_LAYOUT = [{'header': [],
  'indexColumns': [1],
  'primary_symbol': 'pCarbonPrice',
  'relative_path': 'data_test/constraint/pCarbonPrice.csv',
  'symbols': ['pCarbonPrice'],
  'type': 'par',
  'valueColumns': [2]},
 {'header': [1],
  'indexColumns': [1],
  'primary_symbol': 'pEmissionsCountry',
  'relative_path': 'data_test/constraint/pEmissionsCountry.csv',
  'symbols': ['pEmissionsCountry'],
  'type': 'par',
  'valueColumns': []},
 {'header': [],
  'indexColumns': [1],
  'primary_symbol': 'pEmissionsTotal',
  'relative_path': 'data_test/constraint/pEmissionsTotal.csv',
  'symbols': ['pEmissionsTotal'],
  'type': 'par',
  'valueColumns': [2]},
 {'header': [1],
  'indexColumns': [1, 2],
  'primary_symbol': 'pMaxFuellimit',
  'relative_path': 'data_test/constraint/pMaxFuellimit.csv',
  'symbols': ['pMaxFuellimit'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1],
  'primary_symbol': 'pAvailabilityH2',
  'relative_path': 'data_test/h2/pAvailabilityH2.csv',
  'symbols': ['pAvailabilityH2'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1],
  'primary_symbol': 'pCapexTrajectoryH2',
  'relative_path': 'data_test/h2/pCapexTrajectoryH2.csv',
  'symbols': ['pCapexTrajectoryH2'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2],
  'primary_symbol': 'pExternalH2',
  'relative_path': 'data_test/h2/pExternalH2.csv',
  'symbols': ['pExternalH2'],
  'type': 'par',
  'valueColumns': []},
 {'header': [],
  'indexColumns': [1],
  'primary_symbol': 'pFuelDataH2',
  'relative_path': 'data_test/h2/pFuelDataH2.csv',
  'symbols': ['pFuelDataH2'],
  'type': 'par',
  'valueColumns': [2]},
 {'header': [1],
  'indexColumns': [1],
  'primary_symbol': 'pH2DataExcel',
  'relative_path': 'data_test/h2/pH2DataExcel.csv',
  'symbols': ['pH2DataExcel'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2, 3, 4],
  'primary_symbol': 'pDemandData',
  'relative_path': 'data_test/load/pDemandData.csv',
  'symbols': ['pDemandData'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2],
  'primary_symbol': 'pDemandForecast',
  'relative_path': 'data_test/load/pDemandForecast.csv',
  'symbols': ['pDemandForecast'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2, 3],
  'primary_symbol': 'pDemandProfile',
  'relative_path': 'data_test/load/pDemandProfile.csv',
  'symbols': ['pDemandProfile'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1],
  'primary_symbol': 'pEnergyEfficiencyFactor',
  'relative_path': 'data_test/load/pEnergyEfficiencyFactor.csv',
  'symbols': ['pEnergyEfficiencyFactor'],
  'type': 'par',
  'valueColumns': []},
 {'header': [],
  'indexColumns': [1],
  'primary_symbol': 'sRelevant',
  'relative_path': 'data_test/load/sRelevant.csv',
  'symbols': ['sRelevant'],
  'type': 'set',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2],
  'primary_symbol': 'pHours',
  'relative_path': 'data_test/pHours.csv',
  'symbols': ['pHours'],
  'type': 'par',
  'valueColumns': []},
 {'header': [],
  'indexColumns': [2],
  'primary_symbol': 'pSettings',
  'relative_path': 'data_test/pSettings.csv',
  'symbols': ['pSettings'],
  'type': 'par',
  'valueColumns': [3]},
 {'header': [],
  'indexColumns': [1],
  'primary_symbol': 'pPlanningReserveMargin',
  'relative_path': 'data_test/reserve/pPlanningReserveMargin.csv',
  'symbols': ['pPlanningReserveMargin'],
  'type': 'par',
  'valueColumns': [2]},
 {'header': [1],
  'indexColumns': [1],
  'primary_symbol': 'pSpinningReserveReqCountry',
  'relative_path': 'data_test/reserve/pSpinningReserveReqCountry.csv',
  'symbols': ['pSpinningReserveReqCountry'],
  'type': 'par',
  'valueColumns': []},
 {'header': [],
  'indexColumns': [1],
  'primary_symbol': 'pSpinningReserveReqSystem',
  'relative_path': 'data_test/reserve/pSpinningReserveReqSystem.csv',
  'symbols': ['pSpinningReserveReqSystem'],
  'type': 'par',
  'valueColumns': [2]},
 {'header': [1],
  'indexColumns': [1],
  'primary_symbol': 'pAvailability',
  'relative_path': 'data_test/supply/pAvailabilityCustom.csv',
  'symbols': ['pAvailability'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2],
  'primary_symbol': 'pCSPData',
  'relative_path': 'data_test/supply/pCSPData.csv',
  'symbols': ['pCSPData'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1],
  'primary_symbol': 'pCapexTrajectories',
  'relative_path': 'data_test/supply/pCapexTrajectoriesCustom.csv',
  'symbols': ['pCapexTrajectories'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2],
  'primary_symbol': 'pFuelPrice',
  'relative_path': 'data_test/supply/pFuelPrice.csv',
  'symbols': ['pFuelPrice'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2, 3, 4],
  'primary_symbol': 'pGenDataInput',
  'relative_path': 'data_test/supply/pGenDataInput.csv',
  'symbols': ['gmap', 'pGenDataInput'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2],
  'primary_symbol': 'pStorageDataInput',
  'relative_path': 'data_test/supply/pStorageDataInput.csv',
  'symbols': ['pStorageDataInput'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2, 3, 4],
  'primary_symbol': 'pVREProfile',
  'relative_path': 'data_test/supply/pVREProfile.csv',
  'symbols': ['pVREProfile'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2, 3],
  'primary_symbol': 'pVREgenProfile',
  'relative_path': 'data_test/supply/pVREgenProfile.csv',
  'symbols': ['pVREgenProfile'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2, 3, 4],
  'primary_symbol': 'pExtTransferLimit',
  'relative_path': 'data_test/trade/pExtTransferLimit.csv',
  'symbols': ['pExtTransferLimit'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2],
  'primary_symbol': 'pLossFactorInternal',
  'relative_path': 'data_test/trade/pLossFactorInternal.csv',
  'symbols': ['pLossFactorInternal'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1],
  'primary_symbol': 'pMaxAnnualExternalTradeShare',
  'relative_path': 'data_test/trade/pMaxAnnualExternalTradeShare.csv',
  'symbols': ['pMaxAnnualExternalTradeShare'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1],
  'primary_symbol': 'pMaxPriceImportShare',
  'relative_path': 'data_test/trade/pMaxPriceImportShare.csv',
  'symbols': ['pMaxPriceImportShare'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2],
  'primary_symbol': 'pMinImport',
  'relative_path': 'data_test/trade/pMinImport.csv',
  'symbols': ['pMinImport'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2],
  'primary_symbol': 'pNewTransmission',
  'relative_path': 'data_test/trade/pNewTransmission.csv',
  'symbols': ['pNewTransmission'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2, 3, 4],
  'primary_symbol': 'pTradePrice',
  'relative_path': 'data_test/trade/pTradePrice.csv',
  'symbols': ['pTradePrice'],
  'type': 'par',
  'valueColumns': []},
 {'header': [1],
  'indexColumns': [1, 2, 3],
  'primary_symbol': 'pTransferLimit',
  'relative_path': 'data_test/trade/pTransferLimit.csv',
  'symbols': ['pTransferLimit'],
  'type': 'par',
  'valueColumns': []},
 {'header': [],
  'indexColumns': [1],
  'primary_symbol': 'zext',
  'relative_path': 'data_test/trade/zext.csv',
  'symbols': ['zext'],
  'type': 'set',
  'valueColumns': []},
 {'header': [],
  'indexColumns': [1],
  'primary_symbol': 'y',
  'relative_path': 'data_test/y.csv',
  'symbols': ['y'],
  'type': 'set',
  'valueColumns': []},
 {'header': [],
  'indexColumns': [1, 2],
  'primary_symbol': 'zcmap',
  'relative_path': 'data_test/zcmap.csv',
  'symbols': ['zcmap'],
  'type': 'set',
  'valueColumns': []}]

LAYOUT_BY_SYMBOL = {}
for entry in CSV_LAYOUT:
    for symbol in entry['symbols']:
        LAYOUT_BY_SYMBOL[symbol] = entry

EXPECTED_SYMBOLS = {entry['primary_symbol'] for entry in CSV_LAYOUT}
OPTIONAL_SYMBOLS = {
    'ftfindex',
    'pAvailability',
    'pAvailabilityDefault',
    'pAvailabilityH2',
    'pCSPData',
    'pCapexTrajectories',
    'pCapexTrajectoriesDefault',
    'pCapexTrajectoryH2',
    'pCarbonPrice',
    'pDemandData',
    'pDemandForecast',
    'pDemandProfile',
    'pEmissionsCountry',
    'pEmissionsTotal',
    'pEnergyEfficiencyFactor',
    'pExtTransferLimit',
    'pExternalH2',
    'pFuelCarbonContent',
    'pFuelDataH2',
    'pFuelPrice',
    'pGenDataInputDefault',
    'pH2DataExcel',
    'pHours',
    'pLossFactorInternal',
    'pMaxFuellimit',
    'pMaxPriceImportShare',
    'pMinImport',
    'pNewTransmission',
    'pPlanningReserveMargin',
    'pSpinningReserveReqCountry',
    'pSpinningReserveReqSystem',
    'pStorageDataInput',
    'pTechData',
    'pTradePrice',
    'pTransferLimit',
    'pVREProfile',
    'pVREgenProfile',
    'zext',
    'sRelevant'
}



## Step 4 · Load the symbol mapping
The mapping table in `input/symbol_mapping.csv` tells the exporter which GDX symbol feeds each legacy CSV. By default every row maps to itself (e.g., `pGenDataInput → pGenDataInput`). When an older GDX uses different names, edit the second column to point to the actual symbol. Optional inputs (such as `pCarbonPrice` or `pEmissionsCountry`) should keep their rows even if they map to missing symbols—the notebook will then write an empty placeholder CSV with the proper name.


In [28]:

symbol_mapping_df = pd.read_csv(MAPPING_PATH)
if {'csv_symbol', 'gdx_symbol'} - set(symbol_mapping_df.columns):
    raise ValueError("symbol_mapping.csv must contain 'csv_symbol' and 'gdx_symbol' columns")

symbol_mapping_df = symbol_mapping_df.fillna('').drop_duplicates(subset='csv_symbol', keep='last')
symbol_mapping = {
    row.csv_symbol: (row.gdx_symbol or row.csv_symbol)
    for row in symbol_mapping_df.itertuples()
}

missing_mapping_rows = [entry['primary_symbol'] for entry in CSV_LAYOUT if entry['primary_symbol'] not in symbol_mapping]
if missing_mapping_rows:
    print("Mapping table missing entries for:", ', '.join(missing_mapping_rows))



## Step 5 · Read the GDX file
Load the legacy GDX once so the symbols are available for export.


In [29]:

container = gt.Container()
container.read(str(GDX_PATH.resolve()))

loaded_symbols = set(container.data.keys())
len(loaded_symbols)


48


## Step 6 · Helper functions
These utilities reshape each symbol so it follows the legacy CSV conventions.


In [30]:

def find_value_column(df: pd.DataFrame) -> Optional[str]:
    """Return the canonical value column name, if present."""
    for candidate in ("value", "Value"):
        if candidate in df.columns:
            return candidate
    return None

def format_set(df: pd.DataFrame) -> pd.DataFrame:
    """Return distinct set elements sorted by all columns."""
    if not list(df.columns):
        return df.copy().reset_index(drop=True)
    return df.drop_duplicates().sort_values(list(df.columns)).reset_index(drop=True)

def format_header_table(df: pd.DataFrame, spec: dict) -> pd.DataFrame:
    """Pivot a parameter that contains header columns into the expected layout."""
    value_col = find_value_column(df)
    domain_cols = [col for col in df.columns if col != value_col]
    index_cols = domain_cols[: len(spec['indexColumns'])]
    header_cols = domain_cols[len(index_cols): len(index_cols) + len(spec['header'])]
    base_cols = index_cols + header_cols + ([value_col] if value_col else [])
    data = df[base_cols] if base_cols else df.copy()
    if header_cols and value_col:
        pivot = data.pivot_table(
            index=index_cols,
            columns=header_cols,
            values=value_col,
            aggfunc='first',
            observed=False,
        )
        pivot.columns = [col if isinstance(col, str) else '_'.join(map(str, col)) for col in pivot.columns]
        return pivot.reset_index().reset_index(drop=True)
    if value_col:
        return data.rename(columns={value_col: 'value'}).reset_index(drop=True)
    return data.reset_index(drop=True)

def format_value_table(df: pd.DataFrame, spec: dict, csv_symbol: str, gdx_symbol: str) -> pd.DataFrame:
    """Return index columns plus the numeric value column."""
    value_col = find_value_column(df)
    if value_col is None:
        raise KeyError(f"No value column found for '{gdx_symbol}' mapped to '{csv_symbol}'")
    domain_cols = [col for col in df.columns if col != value_col]
    index_cols = domain_cols[: len(spec['indexColumns'])]
    columns = index_cols + [value_col]
    return df[columns].reset_index(drop=True)

def build_frame(gdx_symbol: str, csv_symbol: str, spec: dict) -> Optional[pd.DataFrame]:
    """Fetch and reshape a single symbol using the provided spec."""
    if gdx_symbol not in container:
        return None
    records = container[gdx_symbol].records
    if records is None:
        return None
    data = records.copy()
    if spec['type'] == 'set':
        return format_set(data)
    if spec['valueColumns']:
        return format_value_table(data, spec, csv_symbol, gdx_symbol)
    return format_header_table(data, spec)

def fallback_frame(gdx_symbol: str) -> Optional[pd.DataFrame]:
    """Return a plain DataFrame export for symbols without legacy specs."""
    if gdx_symbol not in container:
        return None
    records = container[gdx_symbol].records
    if records is None:
        return None
    return records.copy().reset_index(drop=True)

def empty_frame_from_spec() -> pd.DataFrame:
    """Return an empty DataFrame used for optional placeholders."""
    return pd.DataFrame()



## Step 7 · Write the legacy CSV tree
Iterate over the hard-coded layout first, then place any extra symbols under `extras/`.


In [31]:

summary = []
extras_written = []
skipped = []
missing_in_gdx = []
optional_stubbed = []
empty_in_gdx = []
used_gdx_symbols = set()

for entry in CSV_LAYOUT:
    csv_symbol = entry['primary_symbol']
    gdx_symbol = symbol_mapping.get(csv_symbol, csv_symbol)

    frame = build_frame(gdx_symbol, csv_symbol, entry)
    stubbed_optional = False
    if frame is None:
        if csv_symbol in OPTIONAL_SYMBOLS:
            frame = empty_frame_from_spec()
            stubbed_optional = True
            optional_stubbed.append((csv_symbol, gdx_symbol))
        else:
            missing_in_gdx.append((csv_symbol, gdx_symbol))
            continue
    else:
        frame = frame.copy()

    if not stubbed_optional and frame.empty:
        empty_in_gdx.append((csv_symbol, gdx_symbol))

    relative_path = Path(entry['relative_path'])
    target_path = EXPORT_ROOT / relative_path
    target_path.parent.mkdir(parents=True, exist_ok=True)
    if not OVERWRITE and target_path.exists():
        skipped.append(target_path)
        continue

    frame.to_csv(target_path, index=False, na_rep='')
    summary.append({
        'csv_symbol': csv_symbol,
        'gdx_symbol': gdx_symbol,
        'rows': len(frame),
        'path': target_path.relative_to(EXPORT_ROOT).as_posix(),
    })

    if not stubbed_optional:
        used_gdx_symbols.add(gdx_symbol)
        for alias in entry['symbols']:
            used_gdx_symbols.add(symbol_mapping.get(alias, alias))

extras_candidates = sorted(loaded_symbols - used_gdx_symbols)
for gdx_symbol in extras_candidates:
    frame = fallback_frame(gdx_symbol)
    if frame is None:
        continue
    target_path = EXTRAS_ROOT / f"{gdx_symbol}.csv"
    if not OVERWRITE and target_path.exists():
        skipped.append(target_path)
        continue
    frame.to_csv(target_path, index=False, na_rep='')
    extras_written.append({
        'csv_symbol': '',
        'gdx_symbol': gdx_symbol,
        'rows': len(frame),
        'path': target_path.relative_to(EXPORT_ROOT).as_posix(),
    })

pd.DataFrame(summary + extras_written).sort_values('path')


,csv_symbol,gdx_symbol,rows,path
0,pCarbonPrice,pCarbonPrice,18,data_test/constraint/pCarbonPrice.csv
1,pEmissionsCountry,pEmissionsCountry,0,data_test/constraint/pEmissionsCountry.csv
2,pEmissionsTotal,pEmissionsTotal,29,data_test/constraint/pEmissionsTotal.csv
3,pMaxFuellimit,pMaxFuellimit,0,data_test/constraint/pMaxFuellimit.csv
4,pAvailabilityH2,pAvailabilityH2,12,data_test/h2/pAvailabilityH2.csv
5,pCapexTrajectoryH2,pCapexTrajectoryH2,12,data_test/h2/pCapexTrajectoryH2.csv
6,pExternalH2,pExternalH2,16,data_test/h2/pExternalH2.csv
7,pFuelDataH2,pFuelDataH2,0,data_test/h2/pFuelDataH2.csv
8,pH2DataExcel,pH2DataExcel,12,data_test/h2/pH2DataExcel.csv
9,pDemandData,pDemandData,0,data_test/load/pDemandData.csv



## Step 8 · Review any messages
Understand which symbols were missing, skipped, or written to the fallback `extras/` folder.


In [32]:

if missing_mapping_rows:
    print("Mapping rows missing for:", ', '.join(sorted(missing_mapping_rows)))
if missing_in_gdx:
    formatted = ', '.join(f"{csv} (expected '{gdx}')" for csv, gdx in sorted(missing_in_gdx))
    print("Symbols missing in GDX:", formatted)
if optional_stubbed:
    formatted = ', '.join(f"{csv} (stubbed as '{gdx}')" for csv, gdx in sorted(optional_stubbed))
    print("Optional symbols absent in GDX; wrote empty CSV:", formatted)
if empty_in_gdx:
    formatted = ', '.join(f"{csv} (mapped to '{gdx}')" for csv, gdx in sorted(empty_in_gdx))
    print("Symbols present in GDX but empty:", formatted)
if extras_written:
    print("Extras written:")
    for item in extras_written:
        print(f"  - {item['gdx_symbol']} -> {item['path']}")
if skipped:
    print("Skipped existing files (set OVERWRITE = True to replace them):")
    for path in skipped:
        print(f"  - {path.relative_to(EXPORT_ROOT)}")


Symbols missing in GDX: pMaxAnnualExternalTradeShare (expected 'pMaxAnnualExternalTradeShare')
Optional symbols absent in GDX; wrote empty CSV: pDemandData (stubbed as 'pDemandData'), pEmissionsCountry (stubbed as 'pEmissionsCountry'), pFuelDataH2 (stubbed as 'pFuelDataH2'), pLossFactorInternal (stubbed as 'pLossFactorInternal'), pMaxFuellimit (stubbed as 'pMaxFuellimit'), pMaxPriceImportShare (stubbed as 'pMaxPriceImportShare'), pMinImport (stubbed as 'pMinImport'), pNewTransmission (stubbed as 'pNewTransmission'), pPlanningReserveMargin (stubbed as 'pPlanningReserveMargin'), pSpinningReserveReqCountry (stubbed as 'pSpinningReserveReqCountry'), pVREgenProfile (stubbed as 'pVREgenProfile'), sRelevant (stubbed as 'sRelevant'), zext (stubbed as 'zext')
Extras written:
  - MapGG -> extras/MapGG.csv
  - Relevant -> extras/Relevant.csv
  - ftfindex -> extras/ftfindex.csv
  - hh -> extras/hh.csv
  - pAnnualMaxBuildC -> extras/pAnnualMaxBuildC.csv
  - pAnnualMaxBuildZ -> extras/pAnnualMaxBuil